# In-Context Learning


In-context learning is a generalisation of few-shot learning where the LLM is provided a context as part of the prompt and asked to respond by utilising the information in the context.

* Example: *"Summarize this research article into one paragraph highlighting its strengths and weaknesses: [insert article text]”*
* Example: *"Extract all the quotes from this text and organize them in alphabetical order: [insert text]”*

A very popular technique that you will learn in week 5 called Retrieval-Augmented Generation (RAG) is a form of in-context learning, where:
* a search engine is used to retrieve some relevant information
* that information is then provided to the LLM as context


In this example we download some recent research papers from arXiv papers, extract the text from the PDF files and ask Gemini to summarize the articles as well as provide the main strengths and weaknesses of the papers. Finally we print the summaries to a local html file and as markdown.

In [1]:
!pip install requests bs4 google-generativeai pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 25.4 MB/s eta 0:00:00


In [2]:
import os
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
from urllib.request import urlopen, urlretrieve
from IPython.display import Markdown, display
from pypdf import PdfReader
from datetime import date
from tqdm import tqdm
from google.colab import userdata

In [3]:
API_KEY = os.environ.get("GEMINI_API_KEY")
API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=API_KEY)

We select those papers that have been featured in Hugging Face papers.

In [4]:
BASE_URL = "https://huggingface.co/papers"
page = requests.get(BASE_URL)
soup = BeautifulSoup(page.content, "html.parser")
h3s = soup.find_all("h3")

papers = []

for h3 in h3s:
    a = h3.find("a")
    title = a.text
    link = a["href"].replace('/papers', '')

    papers.append({"title": title, "url": f"https://arxiv.org/pdf{link}"})

Code to extract text from PDFs.

In [5]:
def extract_paper(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text


def extract_pdf(url):
    pdf = urlretrieve(url, "pdf_file.pdf")
    reader = PdfReader("pdf_file.pdf")
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


def printmd(string):
    display(Markdown(string))

In [6]:
LLM = "gemini-2.5-flash"
model = genai.GenerativeModel(LLM)

We use Gemini to summarize the papers.

In [7]:

for paper in tqdm(papers):
    try:
        prompt = (
            f"Summarize the following research article. "
            f"Output the summary ONLY as a Markdown table with two columns: 'Strengths' and 'Weaknesses'. "
            f"Provide 2-3 bullet points for each column. \n\n Content: {extract_pdf(paper['url'])}"
        )

        response = model.generate_content(prompt)
        paper["summary_table"] = response.text
    except Exception as e:
        print(f"Generation failed: {e}")
        paper["summary_table"] = "| Strengths | Weaknesses |\n|---|---|\n| N/A | Paper not available |"

100%|██████████| 14/14 [06:03<00:00, 25.94s/it]


We print the results to a html file.

In [8]:

header = f"""
<html>
<head>
    <title>Daily Dose of AI Research</title>
    <style>
        body {{ font-family: sans-serif; margin: 40px; line-height: 1.6; }}
        table {{ border-collapse: collapse; width: 100%; margin: 10px 0; }}
        th, td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
        th {{ background-color: #f2f2f2; }}
        h2 {{ color: #2c3e50; }}
    </style>
</head>
<body>
    <h1>Daily Dose of AI Research</h1>
    <h4>{date.today()}</h4>
    <p><i>Summaries generated with: {LLM}</i></p>
"""

with open("papers.html", "w", encoding="utf-8") as f:
    f.write(header)

for paper in papers:
    html_content = f"""
    <h2><a href="{paper['url']}">{paper['title']}</a></h2>
    <div>{paper['summary_table']}</div>
    <hr>
    """
    with open("papers.html", "a", encoding="utf-8") as f:
        f.write(html_content)

with open("papers.html", "a", encoding="utf-8") as f:
    f.write("</body></html>")

We can also print the results to this notebook as markdown.

In [9]:
for paper in papers:
    printmd(f"### [{paper['title']}]({paper['url']})")
    printmd(paper["summary_table"])
    printmd("---")

### [Learning to Reason in 4D: Dynamic Spatial Understanding for Vision Language Models](https://arxiv.org/pdf/2512.20557)

| Strengths                                                     | Weaknesses                                                    |
| :----------------------------------------------------------- | :------------------------------------------------------------ |
| - Addresses a critical gap by providing scalable resources and a method for dynamic spatial reasoning (DSR) in Vision Language Models, a known limitation. | - Relies on upstream vision foundation models for geometric cue extraction, whose performance in unconstrained, "in-the-wild" videos can introduce noise or errors into the generated data. |
| - Introduces DSR Suite, an automated pipeline for generating large-scale, diverse question-answer pairs from in-the-wild videos, resulting in DSR-Train for training and a human-refined DSR-Bench for comprehensive evaluation. | - Answers in the dataset are primarily qualitative and trend-based (e.g., larger/smaller, left/right) rather than quantitative, due to the inherent limitations of relative-scale 3D reconstruction from monocular footage. |
| - Proposes the lightweight Geometry Selection Module (GSM) that effectively integrates geometric priors into VLMs, significantly enhancing DSR capabilities while maintaining performance on general video understanding benchmarks. | - Performance on direction prediction subtasks (both absolute and relative) remains significantly lower compared to other DSR capabilities, indicating a persistent challenge in forecasting future object movement. |

---

### [TurboDiffusion: Accelerating Video Diffusion Models by 100-200 Times](https://arxiv.org/pdf/2512.16093)

| Strengths                                                                                             | Weaknesses                                                                                           |
| :-------------------------------------------------------------------------------------------------- | :--------------------------------------------------------------------------------------------------- |
| - Achieves an impressive 100–200x speedup in end-to-end video generation latency on RTX 5090 GPUs.  | - Primary performance claims are specifically validated on a single RTX 5090 GPU, with less detailed quantification for other hardware. |
| - Successfully maintains comparable video quality despite significant acceleration, outperforming existing baselines. | - The current framework's applicability is implicitly limited to the tested video diffusion models, with future work planned to extend it to other paradigms. |
| - Employs a comprehensive, multi-faceted optimization strategy combining low-bit attention, sparse attention, step distillation, and W8A8 quantization. | - The article does not provide objective quantitative metrics (e.g., FID, FVD) or user study results to rigorously support the claim of "comparable video quality." |

...

| Strengths                                                                                                                                                                                             | Weaknesses                                                                                                                                                                            |
| :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| - Addresses a critical gap by focusing on long-horizon software evolution scenarios, moving beyond isolated bug fixes.                                                                 | - Currently limited to Python projects, restricting its applicability to other programming languages.                                                                                  |
| - Tasks are highly realistic and complex, requiring multi-step modifications across an average of 21 files, guided by high-level release notes from mature open-source projects.        | - The benchmark's scale of 48 instances is relatively small, which may limit statistical power for fine-grained comparisons between models.                                          |
| - Introduces "Fix Rate," a soft metric that captures partial progress while penalizing regressions, offering a more nuanced and granular evaluation than binary pass/fail scores. | - Relies on release notes as the sole task specification, potentially missing other real-world evolution scenarios such as undocumented security patches or performance optimizations. |

---

### [LLM Swiss Round: Aggregating Multi-Benchmark Performance via Competitive Swiss-System Dynamics](https://arxiv.org/pdf/2512.21010)

| Strengths | Weaknesses |
|---|---|
| - Addresses fundamental limitations of current LLM evaluation, such as arbitrary benchmark weighting and inability to capture path-dependent performance, which static scoring and simple Elo models fail to do. | - Lacks an objective, universally accepted ground-truth ranking for LLM performance, making definitive external validation challenging and potentially subjective based on the definition of "utility." |
| - Provides a robust and nuanced ranking by simulating a multi-round competitive environment, generating a statistically reliable Expected Win Score (E[Sm]) through Monte Carlo simulation, and enabling Failure Sensitivity Analysis (FSA) to differentiate between 'Robust Generalists' and 'Aggressive Specialists'. | - Direct quantitative comparison with traditional aggregate or static pairwise baseline rankings is difficult due to the CSD framework's novel competitive dynamics and unique metrics, which prioritize resilience and competitive fitness over simple average performance. |
| - Demonstrates superior stability and robustness against extreme score perturbations (e.g., API errors affecting specific benchmarks) compared to simple averaging methods, making it more reliable for real-world deployment scenarios. | - While avoiding arbitrary weighting, the framework's "structural importance" is intrinsically tied to the predefined sequential order of benchmarks. The chosen sequence, though justified, is a critical design decision that shapes the competitive dynamics and thereby influences the final ranking. |

---